In [1]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http

doc_dir = "lecun-2022-path" + '/'
doc_dir = "test" + '/'

doc = convert_files_to_docs(dir_path=doc_dir, split_paragraphs=False)
# clean_func=clean_wiki_text, 
# print(doc)

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [2]:
from haystack.nodes import PreProcessor

processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length= 200,
    split_respect_sentence_boundary=True,
    split_overlap= 50,
    add_page_number=True
)
# docs = [{'content': 'This is a sample content. It contains multiple sentences.'},
        # {'content': 'This is another sample content. It contains multiple sentences.'}]
docs = processor.process( doc )
print(docs[0])
# print(docs)

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

<Document: id=2b8d1f62cd0c79b4892a1e15b3772ced, content='DocumentStore
You can think of the DocumentStore as a database that stores your texts and meta data ...'>
[<Document: {'content': 'DocumentStore\nYou can think of the DocumentStore as a database that stores your texts and meta data and provides them to the Retriever at query time. Learn how to choose the best DocumentStore for your use case and how to use it in a pipeline.\n\nSuggest Edits\n\nUse with Retrievers\n\nBy far the most common way to use a DocumentStore in Haystack is to fetch documents using a Retriever. You provide a DocumentStore as an argument when you initialize a Retriever.\n\nInitialization\nTo use a DocumentStore in a pipeline, you must initialize it first. Initializing a new DocumentStore in Haystack is straightforward. Have a look at the instructions for different types of DocumentStores:\n\nclean_header_footer: Use heuristic to remove footers and headers across different pages by searching for the longest co

In [3]:
# print(docs)

In [4]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(sql_url="sqlite:///"+ doc_dir +"my_db.db", faiss_index_factory_str="Flat")


In [5]:
document_store.write_documents(docs)

Writing Documents:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
    model_format="sentence_transformers",
)

document_store.update_embeddings(retriever)

# document_store.save("my_document_store")

Updating Embedding:   0%|          | 0/2 [00:00<?, ? docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
document_store.save(doc_dir+"my_faiss_index.faiss")